In [1]:
import tensorflow as tf
import random
import time
from datetime import datetime

In [2]:
IMAGE_SIZE = 112
INPUT_SIZE = 96
DST_INPUT_SIZE = 56
NUM_CLASS = 5
NUM_EXAMPLES_PER_EPOCH_FOR_TRAIN = 5000
BATCH_SIZE = 100
LOGDIR='/tmp/data.%s' % datetime.now().isoformat()

print(LOGDIR)

def load(csv_files):
    file_queue = tf.train.string_input_producer(csv_files)
    reader = tf.TextLineReader()
    key, value = reader.read(file_queue)
    filename, label = tf.decode_csv(value, [['path'], [1]])

    label = tf.cast(label, tf.int64)
    label = tf.one_hot(label, depth = NUM_CLASS, on_value = 1.0, off_value = 0.0, axis = -1)
    
    jpeg = tf.read_file(filename)
    image = tf.image.decode_jpeg(jpeg, channels=3)
    image = tf.cast(image, tf.float32)
    image.set_shape([IMAGE_SIZE, IMAGE_SIZE, 3])

    cropsize = random.randint(INPUT_SIZE, INPUT_SIZE + (IMAGE_SIZE - INPUT_SIZE) / 2)
    framesize = INPUT_SIZE + (cropsize - INPUT_SIZE) * 2
    image = tf.image.resize_image_with_crop_or_pad(image, framesize, framesize)
    image = tf.random_crop(image, [cropsize, cropsize, 3])
    image = tf.image.random_flip_left_right(image)
    image = tf.image.random_brightness(image, max_delta=0.8)
    image = tf.image.random_contrast(image, lower=0.8, upper=1.0)
    image = tf.image.random_hue(image, max_delta=0.04)
    image = tf.image.random_saturation(image, lower=0.6, upper=1.4)

    image = tf.image.resize_images(image, (DST_INPUT_SIZE, DST_INPUT_SIZE))
    image = tf.image.per_image_standardization(image)

    min_fraction_of_examples_in_queue = 0.4
    min_queue_examples = int(NUM_EXAMPLES_PER_EPOCH_FOR_TRAIN * min_fraction_of_examples_in_queue)

    images, label_batch, filename = tf.train.shuffle_batch(
       [image, label, filename],
        batch_size=BATCH_SIZE,
        capacity=min_queue_examples + BATCH_SIZE * 3,
        min_after_dequeue=min_queue_examples,
        num_threads=8
    )

    tf.summary.image('image', images, max_outputs=100)
    labels = tf.reshape(label_batch, [BATCH_SIZE, NUM_CLASS])

    
    return (images, labels)

/tmp/data.2017-02-09T14:15:57.113476


In [3]:
def inference(images, keep_prob):
    
    print(images)
    x_image = tf.reshape(images, [-1, DST_INPUT_SIZE, DST_INPUT_SIZE, 3])

    with tf.name_scope('filter1'):
        
        num_filters1 = 32

        W_conv1 = tf.Variable(tf.truncated_normal([5, 5, 3, num_filters1], stddev = 0.1))
        b_conv1 = tf.Variable(tf.constant(0.1, shape=[num_filters1]))
        h_conv1 = tf.nn.conv2d(x_image, W_conv1, strides=[1, 1, 1, 1], padding='SAME')
    
    with tf.name_scope('pool1'):
        h_conv1_cutoff = tf.nn.relu(h_conv1 + b_conv1)
        h_pool1 = tf.nn.max_pool(h_conv1_cutoff, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')
    
    with tf.name_scope('filter2'):
        num_filters2 = 64

        W_conv2 = tf.Variable( tf.truncated_normal([5,5, num_filters1, num_filters2],stddev=0.1)) 
        b_conv2 = tf.Variable(tf.constant(0.1, shape=[num_filters2]))
        h_conv2 = tf.nn.conv2d(h_pool1, W_conv2, strides=[1,1,1,1], padding='SAME')
    
    with tf.name_scope('pool2'):
        h_conv2_cutoff = tf.nn.relu(h_conv2 + b_conv2)
        h_pool2 = tf.nn.max_pool(h_conv2_cutoff, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')
        
    with tf.name_scope('filter3'):
        num_filters3 = 128
        
        W_conv3 = tf.Variable(tf.truncated_normal([5, 5, num_filters2, num_filters3], stddev=0.1))
        b_conv3 = tf.Variable(tf.constant(0.1, shape=[num_filters3]))
        h_conv3 = tf.nn.conv2d(h_pool2, W_conv3, strides=[1, 1, 1, 1], padding='SAME')
        
    with tf.name_scope('pool3'):
        h_conv3_cutoff = tf.nn.relu(h_conv3 + b_conv3)
        h_pool3 = tf.nn.max_pool(h_conv3_cutoff, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')
    
    with tf.name_scope('fc1'):
        w = int(DST_INPUT_SIZE / 8)
        h_pool3_flat = tf.reshape(h_pool3, [-1, w*w*num_filters3])

        num_units1 = w*w*num_filters3
        num_units2 = 1024

        w3 = tf.Variable(tf.truncated_normal([num_units1, num_units2]))
        b3 = tf.Variable(tf.constant(0.1, shape=[num_units2])) 
        hidden3 = tf.nn.relu(tf.matmul(h_pool3_flat, w3) + b3)
        
        hidden3_drop = tf.nn.dropout(hidden3, keep_prob)
        
        w0 = tf.Variable(tf.zeros([num_units2, 5]))
        b0 = tf.Variable(tf.zeros([5]))
        
        tf.summary.histogram('weights_output', w0)
        tf.summary.histogram('biases_output', b0)
        
    with tf.name_scope('sotmax'):
        p = tf.nn.softmax(tf.matmul(hidden3_drop, w0) + b0)
        
    return p

In [4]:
def loss(p, labels):
    loss = -tf.reduce_sum(labels * tf.log(tf.clip_by_value(p, 1e-10, 1.0)))
    
    tf.summary.scalar('loss', loss)
    
    return loss

In [5]:
def accuracy(p, labels):
    correct_prediction = tf.equal(tf.argmax(p, 1), tf.argmax(labels, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    
    tf.summary.scalar('accuracy', accuracy)
    
    return accuracy

In [6]:
def optimizer(loss):
    train_step = tf.train.AdamOptimizer(1e-5).minimize(loss)

    return train_step

In [7]:
with tf.Graph().as_default():
    images, labels = load(['train.csv'])
    keep_prob = tf.placeholder(tf.float32)
    p = inference(images, keep_prob=keep_prob)
    loss = loss(p, labels)
    train_step = optimizer(loss)
    accuracy = accuracy(p, labels)
    
    saver = tf.train.Saver()
    sess = tf.InteractiveSession()
    sess.run(tf.global_variables_initializer())

    tf.train.start_queue_runners(sess)
    
    summary = tf.summary.merge_all()
    summary_writer = tf.summary.FileWriter(LOGDIR, sess.graph)
        
    start_time = time.time()
    for step in range(11):
        
        _, loss_val, acc_val = sess.run([train_step, loss, accuracy], feed_dict={keep_prob: 0.99})

        if step % 10 == 0:
            loss_val, acc_val = sess.run([loss, accuracy], feed_dict={keep_prob: 1.0})
            print('Step: %d, Loss: %.2f, Accuracy: %f' % (step, loss_val, acc_val))
            
            summary_str = sess.run(summary, feed_dict={keep_prob: 1.0})
            summary_writer.add_summary(summary_str, step)

        if step % 100 == 0:
            saver.save(sess, 'cnn-session', global_step=step)
    
    duration = time.time() - start_time

Tensor("shuffle_batch:0", shape=(100, 56, 56, 3), dtype=float32)
Step: 0, Loss: 172.75, Accuracy: 0.390000
Step: 10, Loss: 127.35, Accuracy: 0.520000
